<a href="https://www.kaggle.com/code/notsau/cosine-similarity-for-netflix-recommendation?scriptVersionId=161450080" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd 

# EDA

In [2]:
df=pd.read_csv("/kaggle/input/netflix-movies-and-tv-shows-2021/netflixData.csv")

In [3]:
df.head()

,Show Id,Title,Description,Director,Genres,Cast,Production Country,Release Date,Rating,Duration,Imdb Score,Content Type,Date Added
0,cc1b6ed9-cf9e-4057-8303-34577fb54477,(Un)Well,This docuseries takes a deep dive into the luc...,NaN,Reality TV,NaN,United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
1,e2ef4e91-fb25-42ab-b485-be8e3b23dedb,#Alive,"As a grisly virus rampages a city, a lone man ...",Cho Il,"Horror Movies, International Movies, Thrillers","Yoo Ah-in, Park Shin-hye",South Korea,2020.0,TV-MA,99 min,6.2/10,Movie,"September 8, 2020"
2,b01b73b7-81f6-47a7-86d8-acb63080d525,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...","Sabina Fedeli, Anna Migotto","Documentaries, International Movies","Helen Mirren, Gengher Gatti",Italy,2019.0,TV-14,95 min,6.4/10,Movie,"July 1, 2020"
3,b6611af0-f53c-4a08-9ffa-9716dc57eb9c,#blackAF,Kenya Barris and his family navigate relations...,NaN,TV Comedies,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
4,7f2d4170-bab8-4d75-adc2-197f7124c070,#cats_the_mewvie,This pawesome documentary explores how our fel...,Michael Margolis,"Documentaries, International Movies",NaN,Canada,2020.0,TV-14,90 min,5.1/10,Movie,"February 5, 2020"


In [6]:
df.isnull().sum()

Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64

In [7]:
df.shape

(5967, 13)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 5967 entries, 0 to 5966

Data columns (total 13 columns):

 #   Column              Non-Null Count  Dtype  

---  ------              --------------  -----  

 0   Show Id             5967 non-null   object 

 1   Title               5967 non-null   object 

 2   Description         5967 non-null   object 

 3   Director            3903 non-null   object 

 4   Genres              5967 non-null   object 

 5   Cast                5437 non-null   object 

 6   Production Country  5408 non-null   object 

 7   Release Date        5964 non-null   float64

 8   Rating              5963 non-null   object 

 9   Duration            5964 non-null   object 

 10  Imdb Score          5359 non-null   object 

 11  Content Type        5967 non-null   object 

 12  Date Added          4632 non-null   object 

dtypes: float64(1), object(12)

memory usage: 606.2+ KB


In [9]:
df["Rating"].unique()

array(['TV-MA', 'TV-14', 'TV-G', 'TV-PG', 'TV-Y', 'R', 'PG-13', 'TV-Y7',
       'PG', 'G', nan, 'NC-17'], dtype=object)

# Feature Engineering & Modelling

In [12]:
df.columns

Index(['Show Id', 'Title', 'Description', 'Director', 'Genres', 'Cast',
       'Production Country', 'Release Date', 'Rating', 'Duration',
       'Imdb Score', 'Content Type', 'Date Added'],
      dtype='object')

In [13]:
df=df[['Title', 'Description','Genres','Content Type']]

In [14]:
df.isnull().sum()

Title           0
Description     0
Genres          0
Content Type    0
dtype: int64

In [19]:
df.head()

,Title,Description,Genres,Content Type
0,unwel,This docuseries takes a deep dive into the luc...,Reality TV,tv show
1,aliv,"As a grisly virus rampages a city, a lone man ...","Horror Movies, International Movies, Thrillers",movi
2,annefrank parallel stori,"Through her diary, Anne Frank's story is retol...","Documentaries, International Movies",movi
3,blackaf,Kenya Barris and his family navigate relations...,TV Comedies,tv show
4,catsthemewvi,This pawesome documentary explores how our fel...,"Documentaries, International Movies",movi


In [20]:
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
df["Title"] = df["Title"].apply(clean)
df["Description"] = df["Description"].apply(clean)

[nltk_data] Downloading package stopwords to

[nltk_data]     C:\Users\slymn\AppData\Roaming\nltk_data...

[nltk_data]   Package stopwords is already up-to-date!


In [21]:
df.head()

,Title,Description,Genres,Content Type
0,unwel,docuseri take deep dive lucrat well industri t...,Reality TV,tv show
1,aliv,grisli virus rampag citi lone man stay lock in...,"Horror Movies, International Movies, Thrillers",movi
2,annefrank parallel stori,diari ann frank stori retold alongsid five hol...,"Documentaries, International Movies",movi
3,blackaf,kenya barri famili navig relationship race cul...,TV Comedies,tv show
4,catsthemewvi,pawesom documentari explor felin friend becam ...,"Documentaries, International Movies",movi


In [25]:
from sklearn.feature_extraction import text

netflix = df["Description"].tolist()

bi_tfidf = text.TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
bi_matrix = bi_tfidf.fit_transform(netflix)

uni_tfidf = text.TfidfVectorizer(stop_words="english")
uni_matrix = uni_tfidf.fit_transform(netflix)

from sklearn.metrics.pairwise import cosine_similarity

bi_sim = cosine_similarity(bi_matrix)
uni_sim = cosine_similarity(uni_matrix)

In [26]:
def recommend_netflix(x):
    return ".".join(df["Title"].loc[x.argsort()[-5:-1]])
    
df["netflix_uni"] = [recommend_netflix(x) for x in uni_sim]
df["netflix_bi"] = [recommend_netflix(x) for x in bi_sim]

In [27]:
df['netflix_uni']=df['netflix_uni'].str.replace("_", " ").str.upper().str.strip().str.split("\n")
df["netflix_bi"]=df['netflix_bi'].str.replace("_", " ").str.upper().str.strip().str.split("\n")

In [28]:
df.head()

,Title,Description,Genres,Content Type,netflix_uni,netflix_bi
0,unwel,docuseri take deep dive lucrat well industri t...,Reality TV,tv show,[MILK SYSTEM.ELIZ MATSUNAGA UPON CRIME.RIDE TI...,[ROBBERI WORLD BIGGEST ART HEIST.IMMIGR NATION...
1,aliv,grisli virus rampag citi lone man stay lock in...,"Horror Movies, International Movies, Thrillers",movi,[STRETCH ARMSTRONG BREAKOUT.SLASHER.SOCIAL DIS...,[AALORUKKAM.SOCIAL DISTANC.SLASHER.BERLIN SYND...
2,annefrank parallel stori,diari ann frank stori retold alongsid five hol...,"Documentaries, International Movies",movi,[KINGDOM US.LAST DAY.ARTIST.LIFE AHEAD],[ARTIST.WALT DISNEY ANIM STUDIO SHORT FILM COL...
3,blackaf,kenya barri famili navig relationship race cul...,TV Comedies,tv show,[LAST SUMMER.YES GOD YES.KEVIN HART IM GROWN L...,[PATRIOT ACT HASAN MINHAJ.DIAMOND SKY.REAL ROB...
4,catsthemewvi,pawesom documentari explor felin friend becam ...,"Documentaries, International Movies",movi,[SPOOKLEY CHRISTMA KITTEN.BEGIN LIFE.KITTI LOV...,[TRIXI MATTEL MOVE PART.DIVIN.CAT PEOPL.KITTI ...
